In [1]:
# %load demo.py
import face_recognition
import os
from pathlib import Path
import cv2
import dlib
import numpy as np
from PIL import Image
import easydict
from tensorflow.keras.utils import get_file
from contextlib import contextmanager
from omegaconf import OmegaConf
from src.factory import get_model
import sys
import time
import torch
import utils_sp as utils
from models import gazenet
from mtcnn import FaceDetector
import pymysql
import tensorflow as tf
import requests
import random

pretrained_model = "https://github.com/yu4u/age-gender-estimation/releases/download/v0.6/EfficientNetB3_224_weights.11-3.44.hdf5"
#연령,성별 예측모델 로드
modhash = '6d7f7b7ced093a8b3ef6399163da6ece'
# url1 = "http://192.168.0.16:3001/Ads_img1"
# url2 = "http://192.168.0.16:3001/check1"
conn = pymysql.connect(host='localhost', user='ssafy', password='ssafy', db='adreco', charset='utf8')

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX, #얼굴 인식시
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED) # 네모낳게 얼굴 테두리 선으로 표시
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA) #성별,연령 M,F 20

@contextmanager
def video_capture(num): # 카메라 키는거 
    cap = cv2.VideoCapture(num)
    try:
        yield cap
    finally:
        cap.release()

def yield_images():
    # capture video
    with video_capture(0) as cap:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # 카메라 읽는거
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # 카메라 읽는거
        
        while True:
            # get video frame
            ret, img = cap.read()
            print("xxxx")

            if not ret:
                raise RuntimeError("Failed to capture image")

            yield img
            
def db_insert_clear(ad_num, dic, known_face_encodings, known_face_names): # 20초동안 축적된 정보를 디비에 올리고 비움.
    curs = conn.cursor()
    sql = "INSERT INTO user_info VALUES(NOW(), %s, %s, %s, %s, %s)"
    for k in dic:
        if dic[k]['seetime'] != 0.0:
            data = (str(ad_num), k, int(dic[k]['age']/10)*10, dic[k]['gender'], dic[k]['seetime'])
            curs.execute(sql, data)
    conn.commit()
    dic.clear() # 20초동안 들어간 정보 배열 초기화
    known_face_encodings.clear() # 20초안에 들어온 사용자 인코딩 정보 저장된거 삭제
    known_face_names.clear()     # 얘도 뭐 저장같이 되는건데 삭제
    files = os.listdir('knowns')
    for filename in files:  #얘는 인코딩된 사진들 knowns 폴더에 저장되는데 20초 지나면 전부 삭제
        os.remove('knowns/'+filename)

def main():
    # 시선 추적 모델 로드
    args_gaze = easydict.EasyDict({
        "cpu" : None,
        "weights" : 'models/weights/gazenet.pth' #학습완료 모델 가중치 로드하는것.
    })
    print('Loading MobileFaceGaze model...')
    device = torch.device("cuda:0" if (torch.cuda.is_available() and not args_gaze.cpu) else "cpu")
    # gpu 사용하려면 cuda 사용해야함. 안되면 cpu 사용되는 조건문
    gaze_model = gazenet.GazeNet(device) #시선추적 때 사용할 모델 지정.

    if(not torch.cuda.is_available() and not args_gaze.cpu):
        print('Tried to load GPU but found none. Please check your environment')

    state_dict = torch.load(args_gaze.weights, map_location=device)
    gaze_model.load_state_dict(state_dict) # 여기서 시선추적 모델 로드 됨.
    print('Model loaded using {} as device'.format(device))
    
    gaze_model.eval()
    
    timecount = 0.0 # 20초 세는 변수
    margin = 0.4
    known_face_encodings = []
    known_face_names = []
    person_num = 0 # 인식 사람 번호
    dirname = 'knowns' # 파일명
    dic = {} #배열 같은것.
    frame_count = 0 #프레임 속도 잼.
    
    
    curs = conn.cursor()
#     sql = "SELECT MAX(ad_num) FROM ad_log" # 몇번 째 광고인지
#     curs.execute(sql)
#     rows = curs.fetchone() 
    
    
#     if rows[0] == None:
#         ad_num = 1
#     else: ad_num = rows[0]+1 # 광고번호
    
    ad_num = 1

    face_detector = FaceDetector(device=device) # 얼굴 디텍터(검출)를 변수에 저장
    
    weight_file = get_file("EfficientNetB3_224_weights.11-3.44.hdf5", pretrained_model, cache_subdir="pretrained_models",
                               file_hash=modhash, cache_dir=str(Path("__file__").resolve().parent))
    #가중치 파일을 가지고 오고, 
    
    # for face detection
    detector = dlib.get_frontal_face_detector() #face reconigation 패키지에 있는 디텍터
    
    # load model and weights
    model_name, img_size = Path(weight_file).stem.split("_")[:2]
    img_size = int(img_size)
    cfg = OmegaConf.from_dotlist([f"model.model_name={model_name}", f"model.img_size={img_size}"])
    model = get_model(cfg)
    model.load_weights(weight_file) # 여기까지 하면 모델 생성 완료
    
    image_generator = yield_images() # 영상읽은거
    
    for img in image_generator: # img 변수에 한프레임 담김.
        print("????")
        timer = time.time() # 현재시각
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR -> RGB 변환
        img_h, img_w, _ = np.shape(input_img) # 이미지 높이,너비 가져옴
        print("222")
        # detect faces using dlib detector
        detected = detector(input_img, 1) # 디텍팅
        gaze_detected, landmarks = face_detector.detect(Image.fromarray(input_img)) # gaze_detected는 안씀,landmarks:특징점
        
        faces = np.empty((len(detected), img_size, img_size, 3)) # len은 식별된 사람 수, 얼굴들의 사이즈 * 사이즈만큼 빈 행렬을 잡고 
                                                                 # 그 정보들을 faces라는 변수에 넣을 예정
        count_age_gender = [
                0, 0, 0, 0, 0, 0, #남자 10 ~ 60 각 나이대별 사람수 세려고 0으로 싹다 초기값.
                0, 0, 0, 0, 0, 0 #여자 10 ~ 60
            ]
        
        if len(detected) > 0:
            face_names = []
            z_length = []
            for i, (d, lm) in enumerate(zip(detected, landmarks)): # ex 5명식별시 1명씩 처리하는 for문
                x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height() # 얼굴 하나의 좌표
                xw1 = max(int(x1 - margin * w), 0)
                yw1 = max(int(y1 - margin * h), 0)
                xw2 = min(int(x2 + margin * w), img_w - 1)
                yw2 = min(int(y2 + margin * h), img_h - 1)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2) # 얼굴에 네모 그림
                faces[i] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1], (img_size, img_size)) # faces에 디텍팅된 얼굴들 크기가 다
                                                                                        #다르므로 같은 크기로 맞춰줌.

                #얼굴 인식(번호 매기기)
                face_encoding = face_recognition.face_encodings(input_img, [(y1, x1, y2, x2)])[0] # input_img : 전체 화면
                                                                                            #이 전체 화면에 y1,x1,y2,x2 좌표를 encoding
                if len(known_face_encodings) == 0: # 우리가 알고 있는 얼굴들의 배열에 값이 없을 때
                    min_value = 1.0 # 1.0이 최대값. 이걸로 초기화
                else: # 아는 얼굴이 있을 때
                    distances = face_recognition.face_distance(known_face_encodings, face_encoding) # 내가 아는 얼굴들과 새로 인식된 얼굴들을
                                                                                                    # 검사해서 distance를 구해서 같은사람인지 확인
                    min_value = min(distances) # distance가 작을 수록 비슷한 사람이다라는 뜻.

                name = "Unknown"
                if min_value < 0.30: # 위에서 구한 distance의 최소값인 min_value가 0.30 이하 일때 같은 사람으로 판단.
                    index = np.argmin(distances) # 몇번째 사람인지
                    name = known_face_names[index] # 그사람의 번호를 이름으로 사용
                    
                else: # unkowns 일때 사진 저장
                    name = str(person_num) # 초기값이라면 0임 . 0이라는 이름의 사람이라는뜻.
                    person_num += 1 # 다음 사람을 위해 +1
                    
                    # 얼굴 검출 저장
                    detected_face_img = Image.fromarray(input_img[yw1:yw2, xw1:xw2]) # input_img라는 전체 화면에서 해당 좌표만큼 짤라서
                                                                                     # 이미지 저장.
                    detected_face_img.save('knowns/'+name+'.jpg')
                    known_face_names.append(name) # 이미 알고 있는 배열에 이름(사람 번호) 추가.
                    known_face_encodings.append(face_encoding) # 인코딩도 추가
                    
                # Crop and normalize face Face
                gaze_face, gaze_origin, M  = utils.normalize_face(lm, input_img) # 랜드마크(특징점), 전체 화면

                # Predict gaze
                with torch.no_grad():
                    gaze = gaze_model.get_gaze(gaze_face)
                    gaze = gaze[0].data.cpu()   # 시선추적은 cpu사용함.                         
                print("333")
                # Draw results
                img = cv2.circle(img, gaze_origin, 3, (0, 255, 0), -1) # circle 미간의 점.
                img = utils.draw_gaze(img, gaze_origin, gaze, color=(255,0,0), thickness=2) # draw_gaze : 화살표그린 것.

                z_length.append(utils.zlength(gaze)) # z_length : 화살표 길이
                face_names.append(name) # 인식된 사람 이름 넣기.
                if not name in dic.keys():
                    dic[name] = {} # 4번째사람까지 저장되있는데 만약 name이 5라면 초기화 해주고
                    dic[name]['seetime'] = 0.0 # 쳐다본 시간을 0 으로 초기화
            
            # predict ages and genders of the detected faces
            results = model.predict(faces) # ex)5명 resize해놓은것을 넣어서 각각의 예측 결과를 뽑음.
            predicted_genders = results[0] # 0번인덱스 : 성별
            ages = np.arange(0, 101).reshape(101, 1) # 뒤집기
            predicted_ages = results[1].dot(ages).flatten()
            # results[1] : 나이  / dot:행렬곱  / faltten: 2차원배열 -> 1차원으로 변경
            
            for (i, d), name in zip(enumerate(detected), face_names):
                if not 'age' in dic[name].keys(): #이미 나이를 알고 있다면 안돌게함.
                    age = int(predicted_ages[i])
                    gender =  "M" if predicted_genders[i][0] < 0.5 else "F"
                    if 10 <= age and age <= 69:
                        if gender == 'M': count_age_gender[0 + (int(age/10)-1)] += 1 # 각 나이대별 count증가
                        else: count_age_gender[6 + (int(age/10)-1)] += 1
                            
                    dic[name]['age'], dic[name]['gender'] = age, gender
                label = "{}, {}, {}".format(name, dic[name]['age'], dic[name]['gender']) # 식별사용자 번호, 나이, 성별 라벨로 만듬
                draw_label(img, (d.left(), d.top()), label) #카메라에 담기는 화면 전체 이미지 left,top 만큼 옮긴 좌표에 그림,만든 라벨
            if frame_count == 0: # 첫 프레임일 때
                m = max(count_age_gender)
                m_list = [i for i, j in enumerate(count_age_gender) if j == m]
                choice = random.choice(m_list) # 10대 10명,30대 10명으로 가장 많다면? 둘중에 랜덤으로 뽑음.
                x, y = choice%6, int(choice/6) # 몇번째 열이고 행인지 나옴
                data2 = { 'check_req': True } # 첫번째 프레임일때 광고가 바뀔 수 있게 true로 해둠.
                data1 = {
                    'age': 0 if not count_age_gender else (x+1)*10, 
                    'gender': '0' if not count_age_gender else ("M" if y == 0 else "F")
                }
                #requests.post(url2, json=data2) #true로 넘어가면 광고 바뀔때다.
                #requests.post(url1, json=data1) # 10, F 넘어감.

            addtime =  time.time() - timer # 한 프레임당 시간  여기서 timer은 위에 시작할 때 현재시간 찍어둠.
            for i in range(len(z_length)):
                if z_length[i] <= 25: # 거리가 25 이하면 쳐다보고 있다고 가정
                    dic[face_names[i]]['seetime'] += addtime # 쳐다본 시간(프레임 시간) 만큼 더함.
                img = cv2.putText(img, '{} : {:.2f}'.format(face_names[i], dic[face_names[i]]['seetime']), (0, 40+20*i), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1, cv2.LINE_AA)    
                        #카메라 나오면 화면 왼쪽 상단에 표시되는 사용자 이름(번호), 좌표, 쳐다본시간
        else:
            if frame_count == 0:
                data2 = { 'check_req': True }
                data1 = {
                    'age': 0, 
                    'gender': '0'
                }
                #requests.post(url2, json=data2)
                #requests.post(url1, json=data1)
            addtime =  time.time() - timer
        timecount += addtime;
        img = cv2.putText(img, 'TIME: {:.2f}'.format(timecount), (0, 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1, cv2.LINE_AA) 
        frame_count += 1
        if int(timecount) > 20:
            timecount = 0.0
            person_num = 0
            db_insert_clear(ad_num, dic, known_face_encodings, known_face_names)
            ad_num += 1
            frame_count = 0
            
        cv2.imshow("result", img)
        key = cv2.waitKey(30)
            
        if key == 27:  # ESC
            print("ESC", dic)
            break
            
    conn.close()
    cv2.destroyAllWindows() # 창 닫힘.
        
if __name__ == '__main__':
    try:
        main()
    except:
        print("에러")
        conn.close()
        cv2.destroyAllWindows()
        

Loading MobileFaceGaze model...
Model loaded using cuda:0 as device
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222
333
xxxx
????
222


In [2]:
conda install -c menpo opencv3

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/heung/anaconda3/envs/project

  added / updated specs:
    - opencv3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    opencv3-3.1.0              |           py36_0        37.4 MB  menpo
    ------------------------------------------------------------
                                           Total:        37.4 MB

The following NEW packages will be INSTALLED:

  opencv3            menpo/linux-64::opencv3-3.1.0-py36_0



opencv3-3.1.0        | 37.4 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use 

In [60]:
count_age = [0, 0, 0, 0, 0, 0] # 10 ~ 60
count_gender = [5, 4] # M, F
print((count_age.index(max(count_age))+1)*10)

10


In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8965996722408499740,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12504568892306683093
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1113325568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17662421527034369496
 physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4177782695382151827
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [3]:
!nvidia-smi

Thu Nov 05 13:14:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.09       Driver Version: 457.09       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P3    N/A /  N/A |     64MiB /  2048MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import dlib
dlib.DLIB_USE_CUDA

ModuleNotFoundError: No module named 'dlib'

In [8]:
import requests
url1 = "http://172.30.1.53:3001/Ads_img1"
url2 = "http://172.30.1.53:3001/check1"
data2 = {'check_req': True}
data1 = {'age': 10, 'gender': 'M'}
requests.post(url2, json=data2)
requests.post(url1, json=data1)

<Response [200]>

In [9]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='swp', charset='utf8')
curs = conn.cursor()
sql1 = "SELECT MAX(학번) FROM test1"
sql2 = "SELECT MAX(ad_num) FROM ad_log"
curs.execute(sql2)
rows = curs.fetchone()
if rows[0] == None:
    ad_num = 1
else: ad_num = rows[0]+1
print(ad_num)

41


In [39]:
count = [
    7, 0, 4, 9, 2, 9, #남자 10 ~ 60
    9, 1, 6, 5, 9, 0 #여자 10 ~ 60
]
m = count.index(max(count))
print(m)
x, y = m%6, int(m/6)
print(y)
print((x+1)*10)
print("남자" if y == 0 else "F")

import random
m = max(count)
m_list = [i for i, j in enumerate(count) if j == m]
print(random.choice(m_list))

3
0
40
남자
3


In [1]:
!jupyter nbconvert --to script face.ipynb

[NbConvertApp] Converting notebook face.ipynb to script
[NbConvertApp] Writing 11108 bytes to face.py
